## Lesson 2: Tool Calling
https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/3/tool-calling

In [2]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

OPENAI_MODEL = os.getenv("OPENAI_MODEL")

EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")

In [3]:
# Make asyncio loops work inside Jupyter notebooks which already have loops under the hood
import nest_asyncio

nest_asyncio.apply()

In [18]:
from llama_index.core import Settings
from llama_index.embeddings.jinaai import JinaEmbedding
from llama_index.llms.openai_like import OpenAILike

Settings.embed_model = JinaEmbedding(api_key=EMBEDDING_API_KEY)
Settings.llm = OpenAILike(model=OPENAI_MODEL, temperature=0.01, is_chat_model=True)

#### 1. Define a Simple Tool

In [4]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together"""

    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers"""

    return (x + y) ** 2


add_tool = FunctionTool.from_defaults(fn=add)

mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [7]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model=OPENAI_MODEL, is_chat_model=True)

In [8]:
response = llm.predict_and_call(
    tools=[add_tool, mystery_tool],
    user_msg="Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)

print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: mystery
Action Input: {'x': 2, 'y': 9}
Observation: 121
121


#### 2. Define an Auto-Retrieval Tool

In [9]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["./metagpt.pdf"]).load_data()

In [10]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents=documents)

In [11]:
print(nodes[0].get_content())

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, are complicated through logic inconsistencies due to
cascading hallucinations caused by naively chaining LLMs. Here we introduce
MetaGPT, an innovative meta-programm

In [13]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [19]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts([{"key": "page_label", "value": "2"}]),
)

In [20]:
response = query_engine.query("What are some high-level results of MetaGPT?")

print(response)

MetaGPT has achieved state-of-the-art results in code generation benchmarks, with scores of 85.9% and 87.7% in Pass@1 on HumanEval and MBPP evaluations respectively. It has also demonstrated the ability to handle higher levels of software complexity and offer extensive functionality compared to other frameworks. Notably, in experimental evaluations, MetaGPT has achieved a 100% task completion rate, showcasing the robustness and efficiency of its design.


In [21]:
for node in response.source_nodes:
    print(node.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-10', 'last_modified_date': '2024-05-10'}


##### Define the Auto-Retrieval Tool

In [22]:
from typing import List

from llama_index.core.vector_stores import FilterCondition

In [24]:
def vector_query(query: str, page_numbers: List[str]) -> str:
    """
    Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    """

    metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )

    response = query_engine.query(query)

    return str(response)

In [25]:
vector_query_tool = FunctionTool.from_defaults(name="vector_tool", fn=vector_query)

In [26]:
response = llm.predict_and_call(
    tools=[vector_query_tool],
    user_msg="What are the high-level results of MetaGPT as described on page 2?",
    verbose=True,
)

print(response)

Thought: The question is about the high-level results of MetaGPT described on page 2. I need to use a tool to retrieve this information.
Action: vector_tool
Action Input: {'query': 'high-level results of MetaGPT', 'page_numbers': ['2']}
Observation: MetaGPT has achieved state-of-the-art results in code generation benchmarks, with 85.9% and 87.7% Pass@1 scores on HumanEval and MBPP evaluations, respectively. It has demonstrated the ability to handle higher levels of software complexity and offers extensive functionality. In experimental evaluations, MetaGPT has shown a 100% task completion rate, highlighting its robustness and efficiency in design.
MetaGPT has achieved state-of-the-art results in code generation benchmarks, with 85.9% and 87.7% Pass@1 scores on HumanEval and MBPP evaluations, respectively. It has demonstrated the ability to handle higher levels of software complexity and offers extensive functionality. In experimental evaluations, MetaGPT has shown a 100% task completio

In [27]:
for node in response.source_nodes:
    print(node.metadata)

##### Let's add some other tools!

In [28]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

In [29]:
summary_index = SummaryIndex(nodes=nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True
)

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description="Useful if you want to get a summary of MetaGPT",
)

In [30]:
response = llm.predict_and_call(
    tools=[vector_query_tool, summary_tool],
    user_msg="What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True,
)

print(response)

Thought: The user's question is about comparing MetaGPT and ChatDev, as described on page 8. I need to use a tool to retrieve this information from the document.
Action: vector_tool
Action Input: {'query': 'MetaGPT vs ChatDev', 'page_numbers': ['8']}
Observation: MetaGPT demonstrates superior performance compared to ChatDev in various metrics, particularly on the challenging SoftwareDev dataset. MetaGPT achieves a higher score in executability, with a score of 3.75, which is very close to flawless. It also requires less time to complete tasks, taking only 503 seconds, which is significantly less than ChatDev's running time. In terms of code statistics and the cost of human revision, MetaGPT outperforms ChatDev. Although it uses more tokens overall, MetaGPT is more efficient, needing only 126.5 or 124.3 tokens to generate one line of code, compared to ChatDev's 248.9 tokens. These results emphasize the collaborative benefits of SOPs between multiple agents and showcase MetaGPT's capabil

In [31]:
for node in response.source_nodes:
    print(node.metadata)

In [34]:
response = llm.predict_and_call(
    tools=[vector_query_tool, summary_tool],
    user_msg="What is a summary of this MetaGPT paper?",
    verbose=True,
)

print(response)

Thought: (Implicit) I can answer without any more tools!
Answer: summary_tool
```python
tool_call(input='The MetaGPT paper is a research paper that introduces a new method for training large language models. The method involves using a meta-learning approach to optimize the model\'s hyperparameters and improve its performance on a variety of tasks. The paper presents results from experiments conducted on several benchmark datasets, and demonstrates that the proposed method outperforms other state-of-the-art techniques.')
```
summary_tool
```python
tool_call(input='The MetaGPT paper is a research paper that introduces a new method for training large language models. The method involves using a meta-learning approach to optimize the model\'s hyperparameters and improve its performance on a variety of tasks. The paper presents results from experiments conducted on several benchmark datasets, and demonstrates that the proposed method outperforms other state-of-the-art techniques.')
```
